# Text Evaluation using API

This notebook evaluates all normalized texts (_NOR.txt files) using the deployed evaluation API.

For each folder (POS1, POS2, PRE), it:
1. Reads the consignas CSV to get text metadata
2. Processes each _NOR.txt file by sending it to the API
3. Collects evaluation results (nota and feedback)
4. Generates a CSV with results per folder

## Configuration

In [ ]:
# API Configuration
API_HOST = "https://your-runpod-instance.proxy.runpod.net"  # Update this with your actual RunPod.io URL
API_PORT = "8000"  # Default port
API_BASE_URL = f"{API_HOST}" if API_HOST.startswith('http') else f"http://{API_HOST}:{API_PORT}"

# Folders to process
FOLDERS = ['POS1', 'POS2', 'PRE']
DATA_DIR = 'data'

print(f"API Base URL: {API_BASE_URL}")
print(f"Folders to process: {FOLDERS}")

## Import Required Libraries

In [ ]:
import requests
import pandas as pd
import json
import time
from pathlib import Path
from typing import Dict, List, Optional
import re
from datetime import datetime

## Helper Functions

In [ ]:
def load_consignas_csv(folder_path: Path) -> pd.DataFrame:
    """
    Load the consignas CSV file for a given folder.
    Handles different column name variations (File ID vs FileID, Consigna vs TEXTpost2)
    """
    csv_path = folder_path / 'consignas.csv'
    if not csv_path.exists():
        print(f"Warning: {csv_path} not found")
        return pd.DataFrame()
    
    df = pd.read_csv(csv_path)
    
    # Normalize column names
    if 'File ID' in df.columns:
        df.rename(columns={'File ID': 'FileID'}, inplace=True)
    if 'TEXTpost2' in df.columns:
        df.rename(columns={'TEXTpost2': 'Consigna'}, inplace=True)
    
    return df

def get_nor_files(folder_path: Path) -> List[Path]:
    """
    Get all _NOR.txt files in a folder, sorted by name.
    """
    nor_files = sorted(folder_path.glob('*_NOR.txt'))
    return nor_files

def extract_id_from_filename(filename: str) -> str:
    """
    Extract the ID from a filename like 'POS1_11410001_NOR.txt' -> '11410001'
    """
    match = re.search(r'_(\d+)_NOR\.txt$', filename)
    if match:
        return match.group(1)
    return None

def extract_curso_from_id(text_id: str) -> str:
    """
    Extract the curso (grade level) from the text ID.
    The third character (index 2) of the ID represents the grade level.
    
    Examples:
        '11410003' -> '4' -> '4t ESO'
        '11510082' -> '5' -> '5è ESO'
    """
    if not text_id or len(text_id) < 3:
        return '4t ESO'  # Default fallback
    
    try:
        curso_num = int(text_id[2])
        
        # Catalan ordinal mapping for ESO grades
        curso_ordinals = {
            1: '1r ESO',
            2: '2n ESO',
            3: '3r ESO',
            4: '4t ESO',
            5: '5è ESO',
            6: '6è ESO'
        }
        
        return curso_ordinals.get(curso_num, f'{curso_num}è ESO')
    except (ValueError, IndexError):
        return '4t ESO'  # Default fallback

def read_text_file(file_path: Path) -> str:
    """
    Read and return the content of a text file.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read().strip()
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return ""

## API Interaction Functions

In [ ]:
def submit_evaluation_job(items: List[Dict]) -> Optional[Dict]:
    """
    Submit a batch of texts for evaluation.
    Returns the job information including job_id and stream_url.
    """
    url = f"{API_BASE_URL}/evaluate"
    payload = {"items": items}
    
    try:
        response = requests.post(url, json=payload, timeout=30)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error submitting evaluation job: {e}")
        return None

def stream_results(job_id: str) -> List[Dict]:
    """
    Stream results from the API using Server-Sent Events.
    Returns a list of all evaluation results.
    """
    url = f"{API_BASE_URL}/stream/{job_id}"
    results = []
    
    try:
        response = requests.get(url, stream=True, timeout=300)
        response.raise_for_status()
        
        buffer = ""
        for chunk in response.iter_content(chunk_size=1024, decode_unicode=True):
            if chunk:
                buffer += chunk
                
                # Process complete events
                while '\n\n' in buffer:
                    event_text, buffer = buffer.split('\n\n', 1)
                    
                    # Parse SSE format
                    lines = event_text.strip().split('\n')
                    event_data = {}
                    
                    for line in lines:
                        if line.startswith('event:'):
                            event_data['event'] = line[6:].strip()
                        elif line.startswith('data:'):
                            try:
                                event_data['data'] = json.loads(line[5:].strip())
                            except json.JSONDecodeError:
                                pass
                    
                    # Handle different event types
                    if event_data.get('event') == 'batch_complete':
                        batch_results = event_data.get('data', {}).get('results', [])
                        results.extend(batch_results)
                        
                        # Print progress
                        progress = event_data.get('data', {}).get('progress', {})
                        if progress:
                            print(f"Progress: {progress.get('completed', 0)}/{progress.get('total', 0)} "
                                  f"({progress.get('percentage', 0):.1f}%)")
                    
                    elif event_data.get('event') == 'complete':
                        print("Job completed successfully")
                        break
                    
                    elif event_data.get('event') == 'error':
                        error_msg = event_data.get('data', {}).get('message', 'Unknown error')
                        print(f"Error: {error_msg}")
                        break
        
        return results
        
    except requests.exceptions.RequestException as e:
        print(f"Error streaming results: {e}")
        return results

def evaluate_texts(items: List[Dict]) -> List[Dict]:
    """
    Submit texts for evaluation and wait for results.
    """
    print(f"Submitting {len(items)} texts for evaluation...")
    
    # Submit job
    job_info = submit_evaluation_job(items)
    if not job_info:
        print("Failed to submit evaluation job")
        return []
    
    job_id = job_info.get('job_id')
    print(f"Job submitted with ID: {job_id}")
    print(f"Estimated time: {job_info.get('estimated_time_seconds', 0)} seconds")
    
    # Stream results
    print("Streaming results...")
    results = stream_results(job_id)
    
    print(f"Received {len(results)} results")
    return results

## Process Folders

In [ ]:
def process_folder(folder_name: str, batch_size: int = 10) -> pd.DataFrame:
    """
    Process all _NOR.txt files in a folder.
    Returns a DataFrame with evaluation results.
    """
    print(f"\n{'='*80}")
    print(f"Processing folder: {folder_name}")
    print(f"{'='*80}\n")
    
    folder_path = Path(DATA_DIR) / folder_name
    
    # Load consignas CSV
    consignas_df = load_consignas_csv(folder_path)
    if consignas_df.empty:
        print(f"No consignas.csv found for {folder_name}")
        return pd.DataFrame()
    
    print(f"Loaded {len(consignas_df)} consignas")
    
    # Get all _NOR.txt files
    nor_files = get_nor_files(folder_path)
    print(f"Found {len(nor_files)} _NOR.txt files")
    
    if not nor_files:
        print(f"No _NOR.txt files found in {folder_name}")
        return pd.DataFrame()
    
    # Prepare results storage
    all_results = []
    
    # Process files in batches
    for i in range(0, len(nor_files), batch_size):
        batch_files = nor_files[i:i+batch_size]
        print(f"\nProcessing batch {i//batch_size + 1}/{(len(nor_files)-1)//batch_size + 1} "
              f"({len(batch_files)} files)...")
        
        # Prepare batch items
        batch_items = []
        file_metadata = []  # Store metadata for matching results later
        
        for file_path in batch_files:
            # Extract ID from filename
            text_id = extract_id_from_filename(file_path.name)
            if not text_id:
                print(f"Warning: Could not extract ID from {file_path.name}")
                continue
            
            # Extract curso from ID
            curso = extract_curso_from_id(text_id)
            
            # Read text content
            respuesta = read_text_file(file_path)
            if not respuesta:
                print(f"Warning: Empty file {file_path.name}")
                continue
            
            # Get consigna from CSV
            consigna_row = consignas_df[consignas_df['ID'].astype(str) == text_id]
            if consigna_row.empty:
                print(f"Warning: No consigna found for ID {text_id}")
                consigna = "N/A"
            else:
                consigna = consigna_row.iloc[0]['Consigna']
            
            # Prepare API item
            batch_items.append({
                "id_alumno": text_id,
                "curso": curso,
                "consigna": consigna,
                "respuesta": respuesta
            })
            
            # Store metadata
            file_metadata.append({
                'id': text_id,
                'filename': file_path.name,
                'consigna': consigna,
                'curso': curso
            })
        
        if not batch_items:
            print("No valid items in this batch, skipping...")
            continue
        
        # Evaluate batch
        results = evaluate_texts(batch_items)
        
        # Match results with metadata
        for result in results:
            id_alumno = result.get('id_alumno')
            metadata = next((m for m in file_metadata if m['id'] == id_alumno), None)
            
            if metadata:
                all_results.append({
                    'folder': folder_name,
                    'id': id_alumno,
                    'filename': metadata['filename'],
                    'curso': metadata['curso'],
                    'consigna': metadata['consigna'],
                    'nota': result.get('nota'),
                    'feedback': result.get('feedback')
                })
        
        # Small delay between batches to avoid overwhelming the API
        if i + batch_size < len(nor_files):
            time.sleep(1)
    
    # Create DataFrame
    results_df = pd.DataFrame(all_results)
    
    print(f"\n{'-'*80}")
    print(f"Completed processing {folder_name}")
    print(f"Total results: {len(results_df)}")
    if not results_df.empty:
        print(f"Average nota: {results_df['nota'].mean():.2f}")
        print(f"Nota range: {results_df['nota'].min():.0f} - {results_df['nota'].max():.0f}")
    print(f"{'-'*80}\n")
    
    return results_df

## Main Execution

In [ ]:
# Check API health before starting
try:
    health_url = f"{API_BASE_URL}/health"
    response = requests.get(health_url, timeout=10)
    response.raise_for_status()
    health_data = response.json()
    print(f"API Health Check: {health_data.get('status', 'unknown')}")
    print(f"Model loaded: {health_data.get('model_loaded', False)}")
    print(f"GPU available: {health_data.get('gpu_available', False)}")
    print("\nAPI is ready!\n")
except Exception as e:
    print(f"Warning: Could not check API health: {e}")
    print("Continuing anyway...\n")

In [ ]:
# Process each folder and save results
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

for folder_name in FOLDERS:
    try:
        # Process folder
        results_df = process_folder(folder_name, batch_size=10)
        
        if results_df.empty:
            print(f"No results for {folder_name}, skipping CSV generation\n")
            continue
        
        # Save to CSV
        output_filename = f"results_{folder_name}_{timestamp}.csv"
        output_path = Path(DATA_DIR) / folder_name / output_filename
        results_df.to_csv(output_path, index=False, encoding='utf-8')
        
        print(f"✓ Results saved to: {output_path}\n")
        
        # Display summary statistics
        print(f"Summary for {folder_name}:")
        print(results_df['nota'].describe())
        print("\n")
        
    except Exception as e:
        print(f"Error processing {folder_name}: {e}\n")
        continue

print("\n" + "="*80)
print("All folders processed!")
print("="*80)

## Optional: Combine All Results

In [ ]:
# Combine all results into a single DataFrame
all_results = []

for folder_name in FOLDERS:
    folder_path = Path(DATA_DIR) / folder_name
    csv_files = sorted(folder_path.glob(f"results_{folder_name}_*.csv"))
    
    if csv_files:
        # Get the most recent file
        latest_csv = csv_files[-1]
        df = pd.read_csv(latest_csv)
        all_results.append(df)
        print(f"Loaded {len(df)} results from {latest_csv.name}")

if all_results:
    combined_df = pd.concat(all_results, ignore_index=True)
    combined_output = f"results_all_folders_{timestamp}.csv"
    combined_df.to_csv(combined_output, index=False, encoding='utf-8')
    
    print(f"\n✓ Combined results saved to: {combined_output}")
    print(f"\nTotal results: {len(combined_df)}")
    print(f"\nOverall statistics:")
    print(combined_df.groupby('folder')['nota'].describe())
else:
    print("No results files found to combine")